In [6]:
#1> load the environment + define the path
%reload_ext autoreload
%autoreload 2

#make data file
print( "Takes <30s. However, it might take >1 mins or so if network is busy...")
import glob
import os, glob, shutil
import pandas as pd
from datetime import datetime
import numpy as np
import main_funcs as mfun
import utils_funcs as utils # utils is from Vape - catcher file: 
import matplotlib.pyplot as plt
import pickle
import sys
from LakLabAnalysis.Utility.extract_paq_events import extract_paq_data_frame
from tifffile import TiffFile, imread, imwrite
import imageio.v2 as imageio

# set matlab API
import matlab.engine
eng = matlab.engine.start_matlab()
print('Matlab engine is set correctly.')




Takes <30s. However, it might take >1 mins or so if network is busy...
Matlab engine is set correctly.


In [9]:
# Get the list of recordings
animalList= ['HAT009']
info = mfun.analysis(animalList=animalList)
# display the detected session
print( "Total Session before exclsuion criterias: " +  str(info.recordingList .shape[0]))

###### Lets clean the short and bad performing sessions
# Cleaning step 1: Eliminate short sessions - exclude less than 10 mins
info.recordingList = info.recordingList[info.recordingList['duration'] > 10] # >10 mins
# Cleaning step 2: Eliminate bad performing sessions
info.recordingList = info.recordingList[info.recordingList['performance'] > 60].reset_index(drop=True) # >50% performance

###### Lets See all table
# display the detected session
print( "Total Session after exclsuion criterias: " +  str(info.recordingList .shape[0]))
print(info.recordingList.to_string())

Env: decMaking310
Computer: Huriye Windows
Total Session before exclsuion criterias: 55
Total Session after exclsuion criterias: 35
   animalID recordingDate recordingID          sessionName   twoP                  path imagingTiffFileNames                                   sessionNameWithPath            blockName          experimentDefinition   duration    rigName  performance                      stimType                                                                     analysispathname            filepathname
0    HAT009    2025-06-18           2  2025-06-18_2_HAT009  False  Z:\HAT009\2025-06-18                 None  Z:\HAT009\2025-06-18\2\2025-06-18_2_HAT009_Block.mat  2025-06-18_2_HAT009  Grating2AFC_variableStimSize  27.245117  win-al026    71.428571  Grating2AFC_variableStimSize  C:\Users\Huriye\Documents\code\sideBiasLateralisation\analysis\2025-06-18_HAT009_2\  Z:\HAT009\2025-06-18\2
1    HAT009    2025-06-20           1  2025-06-20_1_HAT009  False  Z:\HAT009\2025-06-20     

In [10]:
# For checking performance over time
# Lets plot sessions in animal folders - 
animalList = np.unique(info.recordingList['animalID'])
for animal in animalList:
    animal_sessions = info.recordingList.loc[info.recordingList['animalID'] == animal, :].reset_index(drop=True)
    # each animal one folder
    save_path = os.path.join(info.analysisPath, animal)
    #create if not exist ( in one line)
    os.makedirs(save_path, exist_ok=True) if not os.path.exists(save_path) else None
    csv_file = os.path.join(save_path, f"{animal}_perf.csv")
    if os.path.exists(csv_file):
        pd_beh = pd.read_csv(csv_file) 
    else:
        pd_beh = pd.DataFrame(columns=['session', 'performance', 'twoP']) 

    # Plot each session 
    for ind, row in animal_sessions.iterrows():
        
        session = row['sessionName']     # adjust column name if different
        twoP = row['twoP']
        save_file = os.path.join(save_path, f"{session}_beh.png")
        if not os.path.exists(save_file):
            try:
                perf = eng.visualiseTrainingGrating2AFC(session, 'Grating2AFC_noTimeline', [], nargout=1)
                fig = eng.gcf()
                eng.saveas(fig, save_file, nargout=0)
                eng.close('all', nargout=0) 
                
            except Exception as e:
                print(f"{session} ---> Error in Matlab getBehaData function - maybe wrong profile?! ")
                perf = np.nan

            # append a new row to pd_beh
            pd_beh.loc[len(pd_beh)] = {
                "session": session[5:10],
                "twoP": twoP,
                "performance": perf
            }

            # update performance in original dataframe
            info.recordingList.loc[
                (info.recordingList['animalID'] == animal) & 
                (info.recordingList['sessionName'] == session),
                'performance'
            ] = perf

    # save pd_beh
    pd_beh.to_csv(csv_file, index=False)

    # Plot summary plot session
    perf_save_file = os.path.join(save_path, f"0behSummary_{animal}.png")
    plt.plot(pd_beh['session'], pd_beh['performance'], label='Performance', color='black')
    # Mark twoP
    mask_twoP = pd_beh['twoP'] == True
    plt.scatter(
        pd_beh.loc[mask_twoP, 'session'], 
        pd_beh.loc[mask_twoP, 'performance'],
        facecolors='none', edgecolors='red', s=50, linewidths=2, label='TwoP'
    )

    plt.title(f"Summary Performance Plot - {animal}")
    plt.ylim(0, 100)  # Set y-axis limits
    plt.axhline(y=50, color='grey', linestyle='--') #add a dash line at 50
    plt.xlabel("Session")
    # lets rotate x labels
    plt.xticks(rotation=60, fontsize=8)
    plt.ylabel("Performance")
    plt.savefig(perf_save_file)
    plt.close()


# End of session loop
print('Finished processing all sessions. Saved:' + info.analysisPath)

Finished processing all sessions. Saved:C:\Users\Huriye\Documents\code\sideBiasLateralisation\analysis
